In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn import tree
from IPython.display import SVG
from graphviz import Source

In [2]:
# load IRS csv
# df_2016 = pd.read_csv('https://www.irs.gov/pub/irs-soi/16zpallagi.csv')
# df_2012 = pd.read_csv("https://www.irs.gov/pub/irs-soi/12zpallagi.csv')
df_2016 = pd.read_csv("16zpallagi.csv")
df_2012 = pd.read_csv("12zpallagi.csv")

In [3]:
# filter based on highest income bracket
df_2016 = df_2016.loc[df_2016['agi_stub'] == 6]

In [4]:
# exclude zipcode 0 and 99999 (not sure what these are)
df_2016 = df_2016[~df_2016.zipcode.isin([0, 99999])]

In [5]:
df_2012 = df_2012.loc[df_2012['AGI_STUB'] == 6]

In [6]:
df_2012 = df_2012[~df_2012.zipcode.isin([0, 99999])]

In [7]:
df_2012 = df_2012[['zipcode', 'N1']]

In [8]:
df_2012.columns = ['zipcode', 'N1_2012']

In [9]:
# join the two tables on zip code
df = df_2016.join(df_2012.set_index('zipcode'), on='zipcode')

In [10]:
df = df[~df['N1_2012'].isnull()]

In [11]:
# load UBS branches csv
branch = pd.read_csv("UBS Branches.csv")

In [12]:
# join the two tables on zip code
ubs_data = branch.join(df.set_index('zipcode'), on='Zip')

In [13]:
# table of missing zip codes in IRS dataset
# missing_irs = ubs_data[ubs_data['STATE'].isnull()]

In [14]:
# save missing table to csv
# header = ["City", "State", "Zip"]
# missing_irs.to_csv('missing_irs.csv', columns = header)

In [15]:
# create working table
ubs_irs = ubs_data[ubs_data['STATE'].notnull()]

In [16]:
# drop irrelevant columns
ubs_irs = ubs_irs.drop(['Zip', 'STATE', 'STATEFIPS', 'agi_stub'], axis=1)

In [17]:
# save table to csv
# ubs_irs.to_csv('ubs_irs.csv', index=False)

In [18]:
# merge City and State columns to create Location
ubs_irs['Location'] = ubs_irs.apply (lambda row: row.City + ", " + row.State,axis=1)

In [19]:
# each row means there is an office at Location
ubs_irs['UBS offices'] = ubs_irs.apply (lambda row: 1,axis=1)

In [20]:
# drop extra columns
ubs_irs = ubs_irs.drop(['City', 'State'], axis=1)

In [21]:
# group by Location and sum the stats
group = ubs_irs.groupby('Location', as_index=False).sum()

In [22]:
# save to csv
# group.to_csv('ubs_irs_linear.csv', index=False)

In [23]:
# exclude Location that has 0 stats
group = group[group['N1_2012'] != 0]
group = group[group['N1'] != 0]

In [24]:
######## Jeremy's suggestions

# Adjusted Gross Income / Number of Returns
def ave_AGI (row):
    return row.A00100 / row.N1

# (Number of returns with ordinary divdends + Number of returns with qualified dividends) / Number of Returns
def ave_div (row):
    return (row.N00600 + row.N00650) / row.N1

# Number of returns with business or professional net income / Number of Returns
def ave_pro_income (row):
    return row.N00900 / row.N1

# Number of returns with self employed (Keogh) retirement plans / Number of Returns
def ave_self_retired (row):
    return row.N03300 / row.N1

# Number of returns with itemized deductions / Number of Returns
def ave_item_deduct (row):
    return row.N04470 / row.N1

# Number of returns with real estate deductions / Number of Returns
def ave_RET (row):
    return row.N18500 / row.N1

# Number of returns with Alternative Minimum Tax / Number of Returns
def ave_AMT (row):
    return row.N09600 / row.N1

# Number of returns with earned income credit / Number of Returns
def ave_EIC (row):
    return row.N59660 / row.N1

# Number of returns with unemployment compensation / Number of Returns
## Does this need to be negative?
def ave_unemployment (row):
    return row.N02300 / row.N1

# Educator expenses amount / Number of returns with educator expenses 
def ave_edu_exp (row):
    if row.N03220 == 0:
        return 0
    else:
        return row.A03220 / row.N03220

######## Charlie's suggestions

# Total Tax Credit Amount / Adjusted Gross Income
def tot_tax_cred (row):
    return row.A07100 / row.A00100
	
# Total Income Amount
def tot_inc (row):
    return row.A02650
	
# Total Taxable Pensions and Annuity Amount
def tot_pens (row):
    return row.A01700

# Total Charitable Contributions Amount
def char_cont (row):
    return row.A19700
	
# Total Net Investment Income Tax
def net_inv (row):
    return row.A85300

# Change in number of filings in top bracket 
def filings_change (row):
    return row.N1 / row.N1_2012 - 1.0

In [25]:
# create new columns
group['ave_div'] = group.apply (lambda row: ave_div (row),axis=1)
group['ave_pro_income'] = group.apply (lambda row: ave_pro_income (row),axis=1)
group['ave_self_retired'] = group.apply (lambda row: ave_self_retired (row),axis=1)
group['ave_item_deduct'] = group.apply (lambda row: ave_item_deduct (row),axis=1)
group['ave_RET'] = group.apply (lambda row: ave_RET (row),axis=1)
group['ave_AMT'] = group.apply (lambda row: ave_AMT (row),axis=1)
group['ave_EIC'] = group.apply (lambda row: ave_EIC (row),axis=1)
group['ave_unemployment'] = group.apply (lambda row: ave_unemployment (row),axis=1)
group['ave_edu_exp'] = group.apply (lambda row: ave_edu_exp (row),axis=1)

group['tot_tax_cred'] = group.apply (lambda row: tot_tax_cred (row),axis=1)
group['tot_inc'] = group.apply (lambda row: tot_inc (row),axis=1)
group['tot_pens'] = group.apply (lambda row: tot_pens (row),axis=1)
group['char_cont'] = group.apply (lambda row: char_cont (row),axis=1)
group['net_inv'] = group.apply (lambda row: net_inv (row),axis=1)
group['filings_change'] = group.apply (lambda row: filings_change (row),axis=1)

In [26]:
# create new working table that only uses the selected features
data = group[['ave_div', 'ave_pro_income', 'ave_self_retired', 'ave_item_deduct', 
              'ave_RET', 'ave_AMT', 'ave_EIC', 'ave_unemployment', 'ave_edu_exp',
              'tot_tax_cred', 'tot_inc', 'tot_pens', 'char_cont', 'net_inv', 'filings_change', 'UBS offices']]

In [27]:
# high weight samples
hw_data = data[data['UBS offices'] > 1]

In [28]:
# low weight samples
lw_data = data[data['UBS offices'] == 1]

In [29]:
# randomly split the low weight samples into train and test set
X_train, X_test = train_test_split(lw_data, test_size=0.30, random_state=42)

In [30]:
# add high weight samples to training data
X_train = X_train.append(hw_data)

In [31]:
# drop UBS offices column
X_test = X_test.drop(['UBS offices'], axis=1)

In [32]:
# create 10-fold for cross validation
kf = KFold(n_splits=10)

# One-Class SVM

In [33]:
# kernel can be linear, rbf, poly, sigmoid
ocsvm = svm.OneClassSVM(nu=0.001, kernel='sigmoid', gamma='auto')

In [34]:
# arrays to store model performance
train_ocsvm = []
cv_ocsvm = []
test_ocsvm = []

# split training set into 10 part, each time the model is trained on 9 parts and cross validated on the remaining part
for k, (train_index, test_index) in enumerate(kf.split(X_train)):
    X, X_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    # sample weights
    sample_weights = X['UBS offices']
    # drop UBS offices column
    X = X.drop(['UBS offices'], axis=1)
    X_cv = X_cv.drop(['UBS offices'], axis=1)
    # scale to mean=0 and stdev=1 based on training set
    scaler = preprocessing.StandardScaler().fit(X)
    # apply scaling function on train, cross validation, and test set
    X_train_n = scaler.transform(X)
    X_cv_norm = scaler.transform(X_cv)
    X_test_norm = scaler.transform(X_test)
    # Demo model's sensitivity: modify weights
    X['tot_inc'] = X.apply(lambda row: row.tot_inc * 2.0, axis=1)
    # fit classifier on train set
    ocsvm.fit(X_train_n, sample_weight = sample_weights)
    # prediction on train set, cross validation, and test set (either 1 or -1)
    y_train = ocsvm.predict(X_train_n)
    y_cv = ocsvm.predict(X_cv_norm)
    y_test = ocsvm.predict(X_test_norm)
    # count the number of wrong classifications
    train_error = y_train[y_train == -1].size
    cv_error = y_cv[y_cv == -1].size
    test_error = y_test[y_test == -1].size
    # model accuracy
    train_accuracy = 1 - train_error / y_train.size
    cv_accuracy = 1 - cv_error / y_cv.size
    test_accuracy = 1 - test_error / y_test.size
    # print
    train_ocsvm.append(train_accuracy)
    cv_ocsvm.append(cv_accuracy)
    test_ocsvm.append(test_accuracy)
    print("[fold {0}] train: {1:.2f}, cv: {2:.2f}, test: {3:.2f}".
          format(k, 100 * train_accuracy, 100 * cv_accuracy, 100 * test_accuracy))

[fold 0] train: 98.36, cv: 100.00, test: 100.00
[fold 1] train: 98.91, cv: 100.00, test: 96.20
[fold 2] train: 97.81, cv: 100.00, test: 98.73
[fold 3] train: 97.81, cv: 100.00, test: 98.73
[fold 4] train: 98.37, cv: 100.00, test: 100.00
[fold 5] train: 99.46, cv: 100.00, test: 100.00
[fold 6] train: 97.83, cv: 100.00, test: 98.73
[fold 7] train: 98.91, cv: 100.00, test: 100.00
[fold 8] train: 97.83, cv: 100.00, test: 97.47
[fold 9] train: 98.91, cv: 95.00, test: 100.00


The mean score and the 95% confidence interval of the score estimate are hence given by:

In [35]:
print("Train accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(train_ocsvm).mean(), 100 * np.asarray(train_ocsvm).std() * 2))

Train accuracy: 98.42 (+/- 1.14)


In [36]:
print("CV accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(cv_ocsvm).mean(), 100 * np.asarray(cv_ocsvm).std() * 2))

CV accuracy: 99.50 (+/- 3.00)


In [37]:
print("Test accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(test_ocsvm).mean(), 100 * np.asarray(test_ocsvm).std() * 2))

Test accuracy: 98.99 (+/- 2.48)


# Isolation Forest

In [38]:
# create a model caller
iso_f = IsolationForest(n_estimators=100,random_state=42, contamination=0.000, behaviour='new')

In [39]:
# arrays to store model performance
train_isof = []
cv_isof = []
test_isof = []

# split training set into 10 part, each time the model is trained on 9 parts and cross validated on the remaining part
for k, (train_index, test_index) in enumerate(kf.split(X_train)):
    X, X_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    # sample weights
    sample_weights = X['UBS offices']
    # drop UBS offices column
    X = X.drop(['UBS offices'], axis=1)
    X_cv = X_cv.drop(['UBS offices'], axis=1)
    # Demo model's sensitivity: modify weights
    X['tot_inc'] = X.apply(lambda row: row.tot_inc * 2.0, axis=1)
    # fit classifier on train set
    iso_f.fit(X, sample_weight = sample_weights)
    # prediction on train set, cross validation, and test set (either 1 or -1)
    y_pred_train = iso_f.predict(X)
    y_pred_cv = iso_f.predict(X_cv)
    y_pred_test = iso_f.predict(X_test)
    # count the number of wrong classifications
    train_error = y_pred_train[y_pred_train == -1].size
    cv_error = y_pred_cv[y_pred_cv == -1].size
    test_error = y_pred_test[y_pred_test == -1].size
    # model accuracy
    train_accuracy = 1 - train_error / y_pred_train.size
    cv_accuracy = 1 - cv_error / y_pred_cv.size
    test_accuracy = 1 - test_error / y_pred_test.size
    # print
    train_isof.append(train_accuracy)
    cv_isof.append(cv_accuracy)
    test_isof.append(test_accuracy)
    print("[fold {0}] train: {1:.2f}, cv: {2:.2f}, test: {3:.2f}".
          format(k, 100 * train_accuracy, 100 * cv_accuracy, 100 * test_accuracy))

[fold 0] train: 100.00, cv: 100.00, test: 100.00
[fold 1] train: 100.00, cv: 100.00, test: 100.00
[fold 2] train: 100.00, cv: 100.00, test: 100.00
[fold 3] train: 100.00, cv: 100.00, test: 100.00
[fold 4] train: 100.00, cv: 100.00, test: 100.00
[fold 5] train: 100.00, cv: 100.00, test: 100.00
[fold 6] train: 100.00, cv: 100.00, test: 100.00
[fold 7] train: 100.00, cv: 100.00, test: 100.00
[fold 8] train: 100.00, cv: 100.00, test: 100.00
[fold 9] train: 100.00, cv: 95.00, test: 100.00


The mean score and the 95% confidence interval of the score estimate are hence given by:

In [40]:
print("Train accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(train_isof).mean(), 100 * np.asarray(train_isof).std() * 2))

Train accuracy: 100.00 (+/- 0.00)


In [41]:
print("CV accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(cv_isof).mean(), 100 * np.asarray(cv_isof).std() * 2))

CV accuracy: 99.50 (+/- 3.00)


In [42]:
print("Test accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(test_isof).mean(), 100 * np.asarray(test_isof).std() * 2))

Test accuracy: 100.00 (+/- 0.00)


In [43]:
# first tree
dt = iso_f.estimators_[0]
print(dt)

ExtraTreeRegressor(criterion='mse', max_depth=8, max_features=1,
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=1952926171,
          splitter='random')


In [44]:
# save first tree structure
# tree.export_graphviz(dt, out_file='dtree.dot', feature_names=X.columns, class_names=True, filled=True)

In [45]:
# save as image
'''
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/Thang/Anaconda3/Library/bin/graphviz/'

graph = Source( tree.export_graphviz(dt, out_file=None, feature_names=X.columns, class_names=True, filled=True))
png_bytes = graph.pipe(format='png')
with open('dtree.png','wb') as f:
    f.write(png_bytes)
'''
# from IPython.display import Image
# Image(png_bytes)

'\nimport os\nos.environ["PATH"] += os.pathsep + \'C:/Users/Thang/Anaconda3/Library/bin/graphviz/\'\n\ngraph = Source( tree.export_graphviz(dt, out_file=None, feature_names=X.columns, class_names=True, filled=True))\npng_bytes = graph.pipe(format=\'png\')\nwith open(\'dtree.png\',\'wb\') as f:\n    f.write(png_bytes)\n'

In [46]:
# display in Jupyter Notebook
'''
graph = Source( tree.export_graphviz(dt, out_file=None, feature_names=X.columns, class_names=True, filled=True))
SVG(graph.pipe(format='svg'))
'''

"\ngraph = Source( tree.export_graphviz(dt, out_file=None, feature_names=X.columns, class_names=True, filled=True))\nSVG(graph.pipe(format='svg'))\n"

# Local Outlier Factor

In [47]:
# metric can be cityblock, cosine, euclidean, l1’, l2, manhattan
lof = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination= 0.001, metric='euclidean')

In [48]:
# arrays to store model performance
cv_lof = []
test_lof = []

# split training set into 10 part, each time the model is trained on 9 parts and cross validated on the remaining part
for k, (train_index, test_index) in enumerate(kf.split(X_train)):
    X, X_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    # drop UBS offices column
    X = X.drop(['UBS offices'], axis=1)
    X_cv = X_cv.drop(['UBS offices'], axis=1)
    # scale to mean=0 and stdev=1 based on training set
    scaler = preprocessing.StandardScaler().fit(X)
    # apply scaling function on train, cross validation, and test set
    X_train_n = scaler.transform(X)
    X_cv_norm = scaler.transform(X_cv)
    X_test_norm = scaler.transform(X_test)
    # Demo model's sensitivity: modify weights
    X['tot_inc'] = X.apply(lambda row: row.tot_inc * 2.0, axis=1)
    # fit classifier on train set
    lof.fit(X_train_n)
    # prediction on cross validation and test set (either 1 or -1)
    y_cv = lof.predict(X_cv_norm)
    y_test = lof.predict(X_test_norm)
    # count the number of wrong classifications
    cv_error = y_cv[y_cv == -1].size
    test_error = y_test[y_test == -1].size
    # model accuracy
    cv_accuracy = 1 - cv_error / y_cv.size
    test_accuracy = 1 - test_error / y_test.size
    # print
    cv_lof.append(cv_accuracy)
    test_lof.append(test_accuracy)
    print("[fold {0}] cv: {1:.2f}, test: {2:.2f}".
          format(k, 100 * cv_accuracy, 100 * test_accuracy))

[fold 0] cv: 100.00, test: 100.00
[fold 1] cv: 100.00, test: 100.00
[fold 2] cv: 100.00, test: 100.00
[fold 3] cv: 100.00, test: 100.00
[fold 4] cv: 95.00, test: 100.00
[fold 5] cv: 100.00, test: 100.00
[fold 6] cv: 100.00, test: 100.00
[fold 7] cv: 95.00, test: 100.00
[fold 8] cv: 100.00, test: 100.00
[fold 9] cv: 90.00, test: 100.00


The mean score and the 95% confidence interval of the score estimate are hence given by:

In [49]:
print("CV accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(cv_lof).mean(), 100 * np.asarray(cv_lof).std() * 2))

CV accuracy: 98.00 (+/- 6.63)


In [50]:
print("Test accuracy: %0.2f (+/- %0.2f)" % (100 * np.asarray(test_lof).mean(), 100 * np.asarray(test_lof).std() * 2))

Test accuracy: 100.00 (+/- 0.00)


# Prediction

In [51]:
# create list of Zip codes that do not have UBS office
prediction_set = df[~df.zipcode.isin(branch['Zip'])]

In [52]:
# load zipcode csv
zip_to_city = pd.read_csv("zip_code_database.csv")

In [53]:
zip_to_city = zip_to_city[['zip', 'primary_city']]

In [54]:
# join the two tables on zip code
prediction_matrix = prediction_set.join(zip_to_city.set_index('zip'), on='zipcode')

In [55]:
# merge City and State columns to create Location
# cross_check = prediction_matrix
# cross_check['Location'] = cross_check.apply (lambda row: row.primary_city + ", " + row.STATE,axis=1)

In [56]:
# cross_check = cross_check.drop(['primary_city'], axis=1)

In [57]:
# get a list of columns
# cols = list(cross_check)

# move the column to head of list using index, pop and insert
# cols.insert(0, cols.pop(cols.index('Location')))

# use loc to reorder
# cross_check = cross_check.loc[:, cols]

# save to csv
# cross_check.to_csv('cross_check.csv', index=False)

In [58]:
# drop irrelevant columns
prediction_matrix = prediction_matrix.drop(['zipcode', 'STATEFIPS', 'agi_stub'], axis=1)

In [59]:
# merge City and State columns to create Location
prediction_matrix['Location'] = prediction_matrix.apply (lambda row: row.primary_city + ", " + row.STATE,axis=1)

In [60]:
# get a list of columns
cols = list(prediction_matrix)

# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Location')))

# use loc to reorder
prediction_matrix = prediction_matrix.loc[:, cols]

In [61]:
# drop extra columns
prediction_matrix = prediction_matrix.drop(['primary_city', 'STATE'], axis=1)

In [62]:
# group by Location and sum the stats
group_predict = prediction_matrix.groupby('Location', as_index=False).sum()

In [63]:
# save to csv
# group_predict.to_csv('prediction_set.csv', index=False)

In [64]:
# exclude Location that has 0 stats
group_predict = group_predict[group_predict['N1'] != 0]
group_predict = group_predict[group_predict['N1_2012'] != 0]

In [65]:
# create new columns
group_predict['ave_div'] = group_predict.apply (lambda row: ave_div (row),axis=1)
group_predict['ave_pro_income'] = group_predict.apply (lambda row: ave_pro_income (row),axis=1)
group_predict['ave_self_retired'] = group_predict.apply (lambda row: ave_self_retired (row),axis=1)
group_predict['ave_item_deduct'] = group_predict.apply (lambda row: ave_item_deduct (row),axis=1)
group_predict['ave_RET'] = group_predict.apply (lambda row: ave_RET (row),axis=1)
group_predict['ave_AMT'] = group_predict.apply (lambda row: ave_AMT (row),axis=1)
group_predict['ave_EIC'] = group_predict.apply (lambda row: ave_EIC (row),axis=1)
group_predict['ave_unemployment'] = group_predict.apply (lambda row: ave_unemployment (row),axis=1)
group_predict['ave_edu_exp'] = group_predict.apply (lambda row: ave_edu_exp (row),axis=1)

group_predict['tot_tax_cred'] = group_predict.apply (lambda row: tot_tax_cred (row),axis=1)
group_predict['tot_inc'] = group_predict.apply (lambda row: tot_inc (row),axis=1)
group_predict['tot_pens'] = group_predict.apply (lambda row: tot_pens (row),axis=1)
group_predict['char_cont'] = group_predict.apply (lambda row: char_cont (row),axis=1)
group_predict['net_inv'] = group_predict.apply (lambda row: net_inv (row),axis=1)
group_predict['filings_change'] = group_predict.apply (lambda row: filings_change (row),axis=1)

In [66]:
group_predict = group_predict.reset_index(drop=True)

In [67]:
# create new working table that only uses the selected features
private_set = group_predict[['ave_div', 'ave_pro_income', 'ave_self_retired', 'ave_item_deduct',
                             'ave_RET', 'ave_AMT', 'ave_EIC', 'ave_unemployment', 'ave_edu_exp',
                             'tot_tax_cred', 'tot_inc', 'tot_pens', 'char_cont', 'net_inv', 'filings_change']]

# Results from One-Class SVM

In [68]:
# scale to mean=0 and stdev=1 based on training set
scaler = preprocessing.StandardScaler().fit(X)

# apply scaling function on prediction set
X_predict = scaler.transform(private_set)

# prediction on prediction set (either 1 or -1)
ocsvm_prediction = ocsvm.predict(X_predict)

C:\Users\Thang\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [69]:
# Signed distance to the separating hyperplane (positive for an inlier and negative for an outlier)
result = ocsvm.decision_function(X_predict)

In [70]:
# sort from largest to smallest distance
ocsvm_array = sorted(result, reverse=True)

In [71]:
# top 10
ocsvm_pred = ocsvm_array[0:10]
ocsvm_pred

[0.035271529429985536,
 0.031171973982380828,
 0.030810462096946021,
 0.030507116240225118,
 0.030379213018699114,
 0.030362403698022716,
 0.029626612178356992,
 0.029491174805242829,
 0.029248571565102542,
 0.02923466762881987]

In [72]:
# look up the index of matching distance in the original result array
city_index = []

for idx in ocsvm_pred:
    item_index = np.where(result == idx)
    for l in item_index:
        city_index.extend(l)

In [73]:
# look up Location based on above indices
ocsvm_res = []

for res in city_index:
    ocsvm_res.append(group_predict['Location'][res])
    print(group_predict['Location'][res])

Lakeport, CA
Venice, CA
Milpitas, CA
West Hollywood, CA
Dublin, CA
Trumansburg, NY
Playa Del Rey, CA
South San Francisco, CA
Thorndale, TX
Poteet, TX


# Results from Isolation Forest

In [74]:
# prediction on prediction set (either 1 or -1)
isof_prediction = iso_f.predict(private_set)

In [75]:
# Average anomaly score of X of the base classifiers (higher for normal, lower for outlier)
isof_result = iso_f.decision_function(private_set)

In [76]:
# sort from largest to smallest distance
isof_array = sorted(isof_result, reverse=True)

In [77]:
# top 10
isof_pred = isof_array[0:10]
isof_pred

[0.24927471996893436,
 0.24748397262213306,
 0.24683230855088834,
 0.24667762645447205,
 0.24613895586236395,
 0.24612109122606823,
 0.2459798109802982,
 0.24506685911878467,
 0.24469728986667971,
 0.2446118941448534]

In [78]:
# look up the index of matching distance in the original result array
isof_city_index =[]

for idx in isof_pred:
    item_index = np.where(isof_result == idx)
    for l in item_index:
        isof_city_index.extend(l)

In [79]:
# look up Location based on above indices
iso_f_res = []

for res in isof_city_index:
    iso_f_res.append(group_predict['Location'][res])
    print(group_predict['Location'][res])

Lewisburg, PA
Manheim, PA
Butte, MT
Chippewa Falls, WI
Michigan City, IN
Greenwood, SC
Versailles, KY
Long Branch, NJ
Newark, OH
Great Falls, MT


# Results from Local Outlier Factor

In [80]:
# scale to mean=0 and stdev=1 based on training set
scaler = preprocessing.StandardScaler().fit(X)

# apply scaling function on prediction set
X_predict = scaler.transform(private_set)

# prediction on prediction set (either 1 or -1)
lof_prediction = lof.predict(X_predict)

C:\Users\Thang\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [81]:
# Bigger is better, i.e. large values correspond to inliers.
lof_result = lof.decision_function(X_predict)

In [82]:
# sort from largest to smallest distance
lof_array = sorted(lof_result, reverse=True)

In [83]:
# top 10
lof_pred = lof_array[0:10]
lof_pred

[2.0541312712398025,
 2.0497191999436595,
 2.048284564159327,
 2.0454457212194819,
 2.0445912565853366,
 2.0444879859709832,
 2.0420881400062552,
 2.041885239714472,
 2.0416908299287391,
 2.0415795998818602]

In [84]:
lof_city_index =[]

for idx in lof_pred:
    item_index = np.where(lof_result == idx)
    for l in item_index:
        lof_city_index.extend(l)

In [85]:
lof_res = []

for res in lof_city_index:
    lof_res.append(group_predict['Location'][res])
    print(group_predict['Location'][res])

Huntingdon Valley, PA
Hopkins, MN
Zionsville, IN
Evergreen, CO
Powell, OH
Butte, MT
Cedar Rapids, IA
Western Springs, IL
Kennett Square, PA
Lewisville, NC


# Combined results

In [86]:
combined_results = np.array([result, isof_result, lof_result])

In [87]:
combined_results_df = pd.DataFrame({'OCSVM':combined_results[0,:],'ISOF':combined_results[1,:], 'LOF':combined_results[2,:]})

In [88]:
scaler = preprocessing.MinMaxScaler(feature_range=(0, len(combined_results_df))).fit(combined_results_df)
combined_results_df_n = pd.DataFrame(scaler.transform(combined_results_df), columns = combined_results_df.columns, index = combined_results_df.index)

In [89]:
combined_results_df_n['combined'] = combined_results_df_n.apply (lambda row: row.OCSVM + row.ISOF + row.LOF,axis=1)

In [90]:
combined_array = sorted(combined_results_df_n['combined'], reverse=True)

In [91]:
combined_pred = combined_array[0:10]

In [92]:
combined_pred

[29395.734883890931,
 29338.939444744268,
 29297.837187732031,
 29297.73391589924,
 29290.784378812888,
 29203.379546450058,
 29195.931940814855,
 29188.43427283458,
 29178.508001017406,
 29177.055314281752]

In [93]:
comb_index =[]

for rank in combined_pred:
    idx = combined_results_df_n[combined_results_df_n['combined'] == rank].index.tolist()
    comb_index.extend(idx)

In [94]:
comb_res = []

for res in comb_index:
    comb_res.append(group_predict['Location'][res])
    print(group_predict['Location'][res])

Coralville, IA
Covington, KY
Long Branch, NJ
Closter, NJ
Carmel Valley, CA
Delafield, WI
Lewisburg, PA
Butte, MT
Chippewa Falls, WI
Carpinteria, CA


# Cross checking

In [95]:
# load IRS csv
# cc = pd.read_csv("cross_check.csv")

In [96]:
# Cross check statistics from OCSVM results
# cc_ocsvm = cc[cc['Location'].isin(ocsvm_res)]
# cc_ocsvm

In [97]:
# Cross check statistics from Isolation Forest results
# cc_isof = cc[cc['Location'].isin(iso_f_res)]
# cc_isof

In [98]:
# Cross check statistics from Local Outlier Factor results
# cc_lof = cc[cc['Location'].isin(lof_res)]
# cc_lof

In [99]:
# Cross check statistics from Combined results
# cc_comb = cc[cc['Location'].isin(comb_res)]
# cc_comb

In [100]:
# cc_ocsvm.to_csv('cc_ocsvm.csv', index=False)
# cc_isof.to_csv('cc_isof.csv', index=False)
# cc_lof.to_csv('cc_lof.csv', index=False)
# cc_comb.to_csv('cc_comb.csv', index=False)